In [1]:
import logging
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers, launch_es
from haystack.nodes import FARMReader, TransformersReader, BM25Retriever
import requests

d:\data-science-projects\question-answering-research\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [2]:
# Connect to Elasticsearch
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="sampledata")

In [3]:
doc_dir = "../data/sampleData"
docs = convert_files_to_docs(dir_path=doc_dir, clean_func=clean_wiki_text, split_paragraphs=True)
document_store.write_documents(docs)

INFO - haystack.utils.preprocessing -  Converting ..\data\sampleData\0_Toilet_Training_issues_1.txt
INFO - haystack.utils.preprocessing -  Converting ..\data\sampleData\1_Toilet_Training_issues_2.txt
INFO - haystack.utils.preprocessing -  Converting ..\data\sampleData\2_Toilet_Training_issues_3.txt
INFO - haystack.utils.preprocessing -  Converting ..\data\sampleData\3_Toilet_Training_issues_4.txt
INFO - haystack.utils.preprocessing -  Converting ..\data\sampleData\4_Picky_Eaters_1.txt
INFO - haystack.utils.preprocessing -  Converting ..\data\sampleData\5_Picky_Eaters_2.txt
INFO - haystack.utils.preprocessing -  Converting ..\data\sampleData\6_Picky_Eaters_3.txt


In [4]:
requests.get('http://localhost:9200/sampledata/_count').json()

{'count': 13,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [5]:
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore
from haystack.retriever.dense import DensePassageRetriever

In [6]:
doc_store = ElasticsearchDocumentStore(
    host='localhost',
    username='', password='',
    index='sampledata'
)

In [7]:
retriever = DensePassageRetriever(
    document_store=doc_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-question_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-question_encoder-single-nq-base
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  ======

In [8]:
doc_store.update_embeddings(retriever=retriever)

INFO - haystack.document_stores.elasticsearch -  Updating embeddings for all 13 docs ...
Updating embeddings: 10000 Docs [00:08, 1206.40 Docs/s]       


In [10]:
prediction=retriever.retrieve("How do I toilet train my child as he keeps wetting himself?")

In [12]:
from pprint import pprint
pprint(prediction)

[<Document: {'content': 'According to the Cleveland Clinic, about 15% of children ages 5 or older actually\nstop wetting the bed each year. So its still perfectly normal for a child to wet the\nbed up until the age of 7. It is often just a stage in their development. And while\ngirls can and do wet the bed, it is 2 times for likely for a boy to wet the bed.\nThere are a few things that can cause bedwetting. Dr. Charles Kwon is a pediatric\nnephrologist, says that the underlying issue is usually a bladder thatâ€™s not\nmatured yet. Other possible causes can be a hormone imbalance, constipation,\nthe child is having a lot of stress and anxiety, they have ADHD, they have a small\nbladder, or there is a family history of childhood bedwetting. Bed wetting in some\ncases could also be a sign of sleep apnea if your child snores a lot or shows other\nsigns of sleep apnea, a UTI (urinary tract infections), or even diabetes. Talking\nwith your childs doctor is always suggested, just in case.\nTh

In [20]:
from haystack.reader import FARMReader
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)


WARNING - haystack -  Object 'FARMReader' is imported through a deprecated path. Please check out the docs for the new import path.
WARNING - haystack -  Object 'FARMReader' is imported through a deprecated path. Please check out the docs for the new import path.
INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find deepset/roberta-base-squad2 locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded deepset/roberta-base-squad2
INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.infer -  Got ya 7 parallel workers to do inference ...
INFO - haystack.model

In [21]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [24]:
prediction = pipe.run(
    query="What are Picky Eaters?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.92 Batches/s]


In [25]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'those who consume an inadequate variety of food through rejection of a substantial amount of food stuffs that are both familiar and unfamiliar', 'type': 'extractive', 'score': 0.7082067728042603, 'context': ' as those who consume an inadequate variety of food through rejection of a substantial amount of food stuffs that are both familiar and unfamiliar. Pi', 'offsets_in_document': [{'start': 28, 'end': 170}], 'offsets_in_context': [{'start': 4, 'end': 146}], 'document_id': '64ca69693c45b668774ed16fa941124a', 'meta': {'name': '4_Picky_Eaters_1.txt'}}>,
             <Answer {'answer': 'Swedish preschoolers of different weight status', 'type': 'extractive', 'score': 0.5047638416290283, 'context': '3.\nPMID: 27889496 Free PMC article.\nPicky eating in Swedish preschoolers of different weight status: application of two new screening cut-offs.\nSandvi', 'offsets_in_document': [{'start': 2130, 'end': 2177}], 'offsets_in_context': [{'start': 52, 'end': 99}], 'd

In [27]:
print_answers(prediction, details="minimum")


Query: What are Picky Eaters?
Answers:
[   {   'answer': 'those who consume an inadequate variety of food through '
                  'rejection of a substantial amount of food stuffs that are '
                  'both familiar and unfamiliar',
        'context': ' as those who consume an inadequate variety of food '
                   'through rejection of a substantial amount of food stuffs '
                   'that are both familiar and unfamiliar. Pi'},
    {   'answer': 'Swedish preschoolers of different weight status',
        'context': '3.\n'
                   'PMID: 27889496 Free PMC article.\n'
                   'Picky eating in Swedish preschoolers of different weight '
                   'status: application of two new screening cut-offs.\n'
                   'Sandvi'},
    {   'answer': 'very restricted diets',
        'context': 'led that those with the food related identity of picky '
                   'eater had very restricted diets and experienced strong '
     